In [1]:
import pandas as pd
import numpy as np

In [48]:
transaction = pd.read_csv(r"C:\Users\DYNABOOK\Desktop\cltv_implementation\cltv_implementation\CLTV_Synthetic_Data\Transactional.csv")
order = pd.read_csv(r"C:\Users\DYNABOOK\Desktop\cltv_implementation\cltv_implementation\CLTV_Synthetic_Data\Orders.csv")

In [6]:
print(transaction.describe())
print(transaction.info())
print(order.describe())
print(order.info())

       transaction_id visit_id user_id order_id        purchase_date  \
count            1046     1046    1046     1046                 1046   
unique           1046     1046     292     1046                 1046   
top            TX2045    V8005   U1734  ORD2045  2025-04-05 03:20:57   
freq                1        1      11        1                    1   

       payment_method  
count            1046  
unique              4  
top            wallet  
freq              281  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1046 entries, 0 to 1045
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   transaction_id  1046 non-null   object
 1   visit_id        1046 non-null   object
 2   user_id         1046 non-null   object
 3   order_id        1046 non-null   object
 4   purchase_date   1046 non-null   object
 5   payment_method  1046 non-null   object
dtypes: object(6)
memory usage: 49.2+ KB
None
          quant

In the data 292 customers made 1046 transactions

In [36]:
print(transaction['purchase_date'].max())
print(transaction['purchase_date'].min())

2025-06-01 17:36:51
2025-03-01 08:41:01


the transactions made from march to june (1-3-25 to 1-6-25), for three months.

In [ ]:
order['total'] = order['total_amount']-order['discount_value']+order['shipping_cost']
ab = order['total'] == order['total_payable']
ab.value_counts()

True     1472
False     608
Name: count, dtype: int64

In [51]:
order['total_payable'] = order['total']
 

In [9]:
transactions_df = transaction.merge(order.groupby('order_id', as_index=False)['total_payable'].sum().rename(columns={'total_payable': 'total_order_amount'}), on='order_id', how='left')
transactions_df.head()

,transaction_id,visit_id,user_id,order_id,purchase_date,payment_method,total_order_amount
0,TX1000,V1018,U1002,ORD1000,2025-03-13 16:48:13,credit_card,485.63
1,TX1001,V1021,U1002,ORD1001,2025-04-26 22:00:27,COD,544.60
2,TX1002,V1023,U1002,ORD1002,2025-05-07 10:06:12,paypal,833.42
3,TX1003,V1027,U1002,ORD1003,2025-04-18 18:53:46,COD,30.75
4,TX1004,V1033,U1003,ORD1004,2025-03-16 20:39:21,COD,201.71


In [60]:
transactions_df['purchase_date'] = pd.to_datetime(transactions_df['purchase_date'])
today = transactions_df['purchase_date'].max() + pd.Timedelta(days=1)

customer_level = transactions_df.groupby('user_id').agg(
    recency=('purchase_date', lambda x: (today - x.max()).days),
    frequency=('purchase_date', 'count'),
    monetary=('total_order_amount', 'sum'),
    last_purchase=('purchase_date', 'max'),
    first_purchase=('purchase_date', 'min')
).reset_index()

customer_level['aov'] = round(customer_level['monetary'] / customer_level['frequency'],2)
customer_level['avg_days_between_orders'] = round((customer_level['last_purchase'] - customer_level['first_purchase']).dt.days / (customer_level['frequency'] - 1),0)
customer_level['avg_days_between_orders'] = customer_level['avg_days_between_orders'].fillna(-1).astype(int)

customer_level


,user_id,recency,frequency,monetary,last_purchase,first_purchase,aov,avg_days_between_orders
0,U1002,26,4,1894.40,2025-05-07 10:06:12,2025-03-13 16:48:13,473.60,18
1,U1003,39,3,635.83,2025-04-23 22:47:37,2025-03-16 20:39:21,211.94,19
2,U1005,25,4,1445.07,2025-05-08 15:12:56,2025-03-14 12:56:55,361.27,18
3,U1007,3,7,1971.81,2025-05-29 21:52:43,2025-03-08 18:29:38,281.69,14
4,U1009,25,4,1819.11,2025-05-08 02:01:39,2025-03-17 19:09:08,454.78,17
...,...,...,...,...,...,...,...,...
287,U1989,54,2,1393.60,2025-04-09 06:34:57,2025-04-06 17:34:55,696.80,2
288,U1991,21,5,2668.98,2025-05-11 20:35:59,2025-03-12 23:56:08,533.80,15
289,U1994,72,2,615.97,2025-03-22 09:01:55,2025-03-14 09:28:25,307.98,7
290,U1995,21,3,1368.31,2025-05-11 23:05:37,2025-03-26 04:06:18,456.10,23


In [ ]:
#quantile binning
customer_level['R_score_1'] = pd.qcut(customer_level['recency'], 5, labels=[5,4,3,2,1]).astype(int)  # Lower recency = better
customer_level['F_score_1'] = pd.qcut(customer_level['frequency'].rank(method='first'), 5, labels=[1,2,3,4,5]).astype(int)
customer_level['M_score_1'] = pd.qcut(customer_level['monetary'], 5, labels=[1,2,3,4,5]).astype(int)
customer_level['RFM_score_1'] = customer_level['R_score_1'] + customer_level['F_score_1']+ customer_level['M_score_1']
customer_level['RFM_1'] = customer_level['R_score_1'].astype(str) + customer_level['F_score_1'].astype(str) + customer_level['M_score_1'].astype(str)
#customer_level.drop(['R_score', 'F_score', 'M_score'], axis=1)
customer_level

,user_id,recency,frequency,monetary,last_purchase,first_purchase,aov,avg_days_between_orders,R_score_1,F_score_1,...,RFM_score_1,RFM_1,segment,segment_1,R_score_2,F_score_2,M_score_2,RFM_score_2,RFM_2,segment_2
0,U1002,26,4,1894.40,2025-05-07 10:06:12,2025-03-13 16:48:13,473.60,18,2,3,...,9,234,Medium,Medium,5,4,5,9,545,Medium
1,U1003,39,3,635.83,2025-04-23 22:47:37,2025-03-16 20:39:21,211.94,19,1,2,...,5,122,Low,Low,4,4,5,5,445,Low
2,U1005,25,4,1445.07,2025-05-08 15:12:56,2025-03-14 12:56:55,361.27,18,2,3,...,8,233,Medium,Medium,5,4,5,8,545,Medium
3,U1007,3,7,1971.81,2025-05-29 21:52:43,2025-03-08 18:29:38,281.69,14,5,5,...,14,554,High,High,5,5,5,14,555,High
4,U1009,25,4,1819.11,2025-05-08 02:01:39,2025-03-17 19:09:08,454.78,17,2,3,...,9,234,Medium,Medium,5,4,5,9,545,Medium
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
287,U1989,54,2,1393.60,2025-04-09 06:34:57,2025-04-06 17:34:55,696.80,2,1,2,...,6,123,Low,Low,4,3,5,6,435,Low
288,U1991,21,5,2668.98,2025-05-11 20:35:59,2025-03-12 23:56:08,533.80,15,3,5,...,13,355,High,High,5,5,5,13,555,High
289,U1994,72,2,615.97,2025-03-22 09:01:55,2025-03-14 09:28:25,307.98,7,1,2,...,5,122,Low,Low,3,3,5,5,335,Low
290,U1995,21,3,1368.31,2025-05-11 23:05:37,2025-03-26 04:06:18,456.10,23,3,3,...,9,333,Medium,Medium,5,4,5,9,545,Medium


In [ ]:

q1 = customer_level['RFM_score_1'].quantile(0.33)
q2 = customer_level['RFM_score_1'].quantile(0.66)

# Apply segmentation
def rfm_level(score):
    if score <= q1:
        return 'Low'
    elif score <= q2:
        return 'Medium'
    else:
        return 'High'

customer_level['segment_1'] = customer_level['RFM_score_1'].apply(rfm_level)

In [70]:
customer_level['segment_1'].value_counts()

segment_1
Medium    106
Low       106
High       80
Name: count, dtype: int64

In [ ]:
#manual binning
recency_max = customer_level['recency'].max()

# Ensure bins are increasing — take max of 121 and actual max
recency_bins = [0, 30, 60, 90, 120, max(121, recency_max + 1)]

customer_level['R_score_2'] = pd.cut(
    customer_level['recency'],
    bins=recency_bins,
    labels=[5, 4, 3, 2, 1],  # lower recency = higher score
    right=False
).astype(int)

frequency_max = customer_level['frequency'].max()
freq_bins = [0, 1, 2, 3, 5, max(6, frequency_max + 1)]

customer_level['F_score_2'] = pd.cut(
    customer_level['frequency'],
    bins=freq_bins,
    labels=[1, 2, 3, 4, 5],
    right=False
).astype(int)

# Monetary
monetary_max = customer_level['monetary'].max()
monetary_bins = [0, 100, 200, 300, 400, max(401, monetary_max + 1)]

customer_level['M_score_2'] = pd.cut(
    customer_level['monetary'],
    bins=monetary_bins,
    labels=[1, 2, 3, 4, 5],
    right=False
).astype(int)

# Combine
customer_level['RFM_score_2'] = customer_level['R_score_1'] + customer_level['F_score_1']+ customer_level['M_score_1']
customer_level['RFM_2'] = customer_level['R_score_2'].astype(str) + customer_level['F_score_2'].astype(str) + customer_level['M_score_2'].astype(str)

In [76]:
q1 = customer_level['RFM_score_2'].quantile(0.33)
q2 = customer_level['RFM_score_2'].quantile(0.66)

# Apply segmentation
def rfm_level(score):
    if score <= q1:
        return 'Low'
    elif score <= q2:
        return 'Medium'
    else:
        return 'High'
customer_level['segment_2'] = customer_level['RFM_score_2'].apply(rfm_level)

In [77]:
customer_level['segment_2'].value_counts()

segment_2
Medium    106
Low       106
High       80
Name: count, dtype: int64

In [ ]:
#RFM Weighting - give importance to frequency, monetory
customer_level['rfm_weighted_score'] = (
    0.3 * customer_level['R_score_1'] + 
    0.4 * customer_level['F_score_1'] + 
    0.3 * customer_level['M_score_1']
)

customer_level['segment_3'] = pd.qcut(customer_level['rfm_weighted_score'], q=3, labels=['Low', 'Medium', 'High'])

In [83]:
customer_level['segment_3'].value_counts()

segment_3
Low       98
High      98
Medium    96
Name: count, dtype: int64

In [84]:
# Percentile RFM scoring
# Step 1: Percentile-Based Scoring
# Higher R_score = more recent, so rank descending
customer_level['R_score_4'] = customer_level['recency'].rank(pct=True, ascending=False) * 5

# Higher F and M scores = better, so rank ascending
customer_level['F_score_4'] = customer_level['frequency'].rank(pct=True) * 5
customer_level['M_score_4'] = customer_level['monetary'].rank(pct=True) * 5

# Step 2: Round scores to 1–5 and clip to avoid 0 or >5
customer_level['R_score_4'] = customer_level['R_score_4'].round(0).astype(int).clip(1, 5)
customer_level['F_score_4'] = customer_level['F_score_4'].round(0).astype(int).clip(1, 5)
customer_level['M_score_4'] = customer_level['M_score_4'].round(0).astype(int).clip(1, 5)

# Step 3: Calculate total RFM score
customer_level['rfm_total_4'] = customer_level['R_score_4'] + customer_level['F_score_4'] + customer_level['M_score_4']

# Step 4: Segment into High, Medium, Low based on quantiles
q1 = customer_level['rfm_total_4'].quantile(0.33)
q2 = customer_level['rfm_total_4'].quantile(0.66)

def segment_customer(score):
    if score <= q1:
        return 'Low'
    elif score <= q2:
        return 'Medium'
    else:
        return 'High'

customer_level['segment_4'] = customer_level['rfm_total_4'].apply(segment_customer)


In [85]:
customer_level['segment_4'].value_counts()

segment_4
Low       113
Medium     95
High       84
Name: count, dtype: int64